In [1]:
import argparse
from pathlib import Path
import sys

import numpy as np
from numpy.lib.recfunctions import structured_to_unstructured
import rospy
# import sensor_msgs.point_cloud2 as pc2
from sensor_msgs.msg import PointCloud2
from jsk_recognition_msgs.msg import BoundingBox, BoundingBoxArray
from visualization_msgs.msg import Marker,MarkerArray
import ros_numpy

from pyquaternion import Quaternion


import torch
import os
import cv2
import pdb

from mmdet3d.apis import inference_detector, init_model, show_result_meshlab
from mmdet3d.core.bbox.structures.box_3d_mode import Box3DMode, LiDARInstance3DBoxes

from copy import deepcopy

from mmdet3d.datasets.pipelines import Compose
from mmcv.parallel import collate, scatter

In [2]:
config = "mmdetection3d/configs/pointpillars/hv_pointpillars_secfpn_6x8_160e_kitti-3d-3class.py"
checkpoint = "mmdetection3d/checkpoints/hv_pointpillars_secfpn_6x8_160e_kitti-3d-3class_20220301_150306-37dc2420.pth"

device = "cuda:0"

data_path = "mmdetection3d/demo/data/kitti/kitti_000008.bin"

model = init_model(config, checkpoint, device=device)


box_type_3d = LiDARInstance3DBoxes
box_mode_3d = Box3DMode.LIDAR


/home/drcl/workspace/Object-Detection-based-Lidar-and-camera/src/detection_pkg/MMLAB_based/mmdetection3d/mmdet3d/models/dense_heads/anchor3d_head.py:84: UserWarning: dir_offset and dir_limit_offset will be depressed and be incorporated into box coder in the future
  warnings.warn(


load checkpoint from local path: mmdetection3d/checkpoints/hv_pointpillars_secfpn_6x8_160e_kitti-3d-3class_20220301_150306-37dc2420.pth


In [3]:
test_pipeline = deepcopy(model.cfg.data.test.pipeline)

In [4]:
test_pipeline = Compose(test_pipeline)

In [16]:
data = dict(
            pts_filename=data_path,
            box_type_3d=box_type_3d,
            box_mode_3d=box_mode_3d,
            # for ScanNet demo we need axis_align_matrix
            ann_info=dict(axis_align_matrix=np.eye(4)),
            sweeps=[],
            # set timestamp = 0
            timestamp=[0],
            img_fields=[],
            bbox3d_fields=[],
            pts_mask_fields=[],
            pts_seg_fields=[],
            bbox_fields=[],
            mask_fields=[],
            seg_fields=[])


data = dict(
            points=[[1,1,1,1]],
            box_type_3d=box_type_3d,
            box_mode_3d=box_mode_3d,
            # for ScanNet demo we need axis_align_matrix
            ann_info=dict(axis_align_matrix=np.eye(4)),
            sweeps=[],
            # set timestamp = 0
            timestamp=[0],
            img_fields=[],
            bbox3d_fields=[],
            pts_mask_fields=[],
            pts_seg_fields=[],
            bbox_fields=[],
            mask_fields=[],
            seg_fields=[])

In [17]:
data

{'points': [[1, 1, 1, 1]],
 'box_type_3d': mmdet3d.core.bbox.structures.lidar_box3d.LiDARInstance3DBoxes,
 'box_mode_3d': <Box3DMode.LIDAR: 0>,
 'ann_info': {'axis_align_matrix': array([[1., 0., 0., 0.],
         [0., 1., 0., 0.],
         [0., 0., 1., 0.],
         [0., 0., 0., 1.]])},
 'sweeps': [],
 'timestamp': [0],
 'img_fields': [],
 'bbox3d_fields': [],
 'pts_mask_fields': [],
 'pts_seg_fields': [],
 'bbox_fields': [],
 'mask_fields': [],
 'seg_fields': []}

In [18]:
data = test_pipeline(data)

KeyError: 'pts_filename'

In [57]:
data = collate([data], samples_per_gpu=1)
data['img_metas'] = data['img_metas'][0].data
data['points'] = data['points'][0].data

In [ ]:
with torch.no_grad():
    result = model(return_loss=False, rescale=True, **data)

In [ ]:
[{'pts_bbox': {'boxes_3d': LiDARInstance3DBoxes(
    tensor([[ 5.2237e+00, -2.3151e+00, -2.4502e+00,  4.4774e+00,  1.8882e+00,
          1.6829e+00,  8.4332e-02,  2.0478e-03,  1.1094e-02],
        [ 5.9916e+00,  7.1663e+00, -2.3601e+00,  5.6326e+00,  2.2297e+00,
          2.1108e+00,  3.1715e+00, -1.1361e-02,  3.8157e-03],
        [ 1.3227e+01, -2.5883e+00, -2.3179e+00,  5.7722e+00,  2.3144e+00,
          2.3388e+00, -1.7291e-02,  3.3550e-03,  1.5071e-03],
        [-3.0663e+01, -3.0841e+01, -2.3407e+00,  4.5754e+00,  1.9888e+00,
          1.6254e+00, -4.9944e-01, -1.0178e-02, -2.9520e-03],
        [ 4.0873e+01, -1.5698e+01, -2.8657e+00,  4.7181e+00,  2.0160e+00,
          1.7874e+00,  1.6673e+00, -7.7149e-03,  6.5610e-03],
        [ 2.6732e+01, -2.2660e+00, -2.3970e+00,  4.2232e+00,  1.8838e+00,
          1.6546e+00, -3.6759e-02, -4.0351e-03,  2.2941e-03],
        [ 1.6685e+01, -2.2745e+00, -2.2343e+00,  4.8004e+00,  1.9957e+00,
          1.8486e+00, -7.3933e-02,  8.0894e-03,  2.8460e-03],
        [ 2.1562e+01,  1.5282e+01, -2.4477e+00,  4.4051e+00,  1.9093e+00,
          1.6224e+00,  5.3027e+00, -7.4279e-03, -6.8408e-03],
        [-3.3877e+01,  2.7628e+01, -2.4291e+00,  4.4412e+00,  1.9621e+00,
          1.6917e+00,  4.3618e-01, -1.7072e-02, -1.3236e-03],
        [ 1.7176e+01, -8.1022e+00, -2.1864e+00,  4.3456e+00,  1.9335e+00,
          1.7067e+00,  4.2234e+00, -9.2177e-03, -3.5142e-03],
        [-3.9341e+01, -2.1991e+01, -1.9452e+00,  4.5767e+00,  1.9854e+00,
          1.6497e+00,  5.0126e-01, -4.0083e-03, -3.3944e-03],
        [ 6.0562e+00,  7.1624e+00, -2.3153e+00,  5.7194e+00,  2.1794e+00,
          2.0985e+00,  3.1214e+00, -4.4809e-02, -1.5617e-02],
        [ 5.2194e+00, -2.4673e+00, -2.4565e+00,  5.2100e+00,  2.1333e+00,
          1.8109e+00,  7.3325e-02, -1.6942e-03,  9.1205e-04],
        [ 1.3227e+01, -2.5883e+00, -2.3179e+00,  5.7722e+00,  2.3144e+00,
          2.3388e+00, -1.7291e-02,  3.3550e-03,  1.5071e-03],
        [ 1.6526e+01, -2.7417e+00, -2.3447e+00,  1.2551e+01,  2.9216e+00,
          3.2800e+00,  3.0144e+00,  9.5163e-03,  5.0941e-03],
        [ 1.1704e+01, -3.2347e+00, -2.3031e+00,  4.6623e+00,  2.0481e+00,
          2.3671e+00,  4.7105e+00, -9.1643e-03,  7.4313e-03],
        [ 1.6224e+01, -2.6326e+00, -2.3062e+00,  1.2536e+01,  2.9532e+00,
          3.1981e+00,  3.0489e+00,  6.4346e-03, -1.9278e-05],
        [ 6.6295e+00,  7.2003e+00, -2.2814e+00,  5.7846e+00,  2.3191e+00,
          2.0696e+00,  3.1286e+00, -2.1029e-05,  2.4841e-04],
        [ 1.2767e+01, -2.6833e+00, -2.2750e+00,  5.1315e+00,  2.1450e+00,
          2.3397e+00, -1.2196e-02, -3.2069e-03, -5.7201e-03],
        [ 5.9616e+00,  7.0335e+00, -2.3538e+00,  5.8159e+00,  2.2134e+00,
          2.0768e+00,  3.0786e+00, -1.5472e-02, -7.4649e-03],
        [ 1.1075e+01, -1.7410e+00, -2.1283e+00,  6.3441e-01,  6.6896e-01,
          1.8446e+00,  3.6886e+00,  6.5583e-03,  1.0465e-03],
        [ 1.4076e+01, -1.3683e+00, -2.3611e+00,  5.6365e-01,  5.4107e-01,
          1.7399e+00,  1.7425e+00, -7.8940e-03, -1.9903e-02],
        [ 1.2181e+01, -1.5895e+00, -2.1659e+00,  6.5683e-01,  6.3523e-01,
          1.8905e+00,  4.7962e+00,  2.8039e-02,  4.8998e-03],
        [ 1.5112e+01, -1.3367e+00, -2.3051e+00,  3.8536e-01,  3.9789e-01,
          1.1325e+00,  1.5778e+00,  7.3007e-03, -5.3488e-03],
        [-2.8528e+01, -3.1049e+01, -2.2539e+00,  3.6901e-01,  3.5743e-01,
          1.0371e+00,  4.4131e+00,  5.2803e-03,  1.5539e-02],
        [ 8.0971e+00,  6.2836e+00, -2.3026e+00,  4.4600e-01,  2.6559e+00,
          1.1789e+00,  4.7847e+00,  1.2471e-02, -9.6804e-03],
        [-3.3547e+01, -2.7003e+01, -2.3000e+00,  5.8158e-01,  3.2024e+00,
          9.7768e-01,  1.0497e+00,  2.0415e-02, -2.4545e-03],
        [-2.9065e+01, -3.0982e+01, -2.2567e+00,  5.9639e-01,  2.8679e+00,
          9.6088e-01,  1.1338e+00,  3.5260e-03, -4.7695e-04],
        [-3.5466e+01, -2.4504e+01, -2.2752e+00,  5.7607e-01,  3.1967e+00,
          9.6746e-01,  1.0323e+00,  1.8307e-03,  8.0396e-03],
        [-2.3610e+01, -3.5488e+01, -2.3129e+00,  5.9840e-01,  3.1519e+00,
          9.0386e-01,  4.2754e+00, -1.1242e-01,  3.0572e-02],
        [-3.1487e+01, -2.9497e+01, -2.2784e+00,  5.7968e-01,  2.9357e+00,
          9.1397e-01,  1.0307e+00,  6.9144e-03,  7.6267e-03],
        [-3.6518e+01, -2.2553e+01, -2.3359e+00,  5.2622e-01,  3.5907e+00,
          9.1186e-01,  4.1768e+00, -1.0759e-03,  9.6183e-03],
        [-2.1095e+01, -3.6982e+01, -2.2859e+00,  5.9599e-01,  3.0859e+00,
          9.5240e-01,  4.2940e+00, -6.7090e-02,  6.1058e-02],
        [-2.5633e+01, -3.4485e+01, -2.2767e+00,  6.0060e-01,  2.9678e+00,
          9.5466e-01,  4.2630e+00, -2.1122e-01,  4.8099e-02],
        [ 2.6073e+01, -3.3990e+01, -2.3901e+00,  5.4623e-01,  2.7263e+00,
          9.4349e-01,  2.2397e+00,  1.1326e-02, -7.2369e-03],
        [-3.3134e+01,  2.7065e+01, -2.4036e+00,  5.5925e-01,  2.9777e+00,
          1.0049e+00,  2.0976e+00,  4.8803e-04, -1.4683e-02],
        [ 2.8543e+01, -3.1961e+01, -2.2903e+00,  5.2068e-01,  2.8363e+00,
          9.3930e-01,  2.0615e+00,  2.5093e-03,  1.2289e-03],
        [ 9.1200e+00,  2.4006e+01, -2.4119e+00,  6.4597e-01,  2.5815e+00,
          9.7685e-01,  4.4074e+00, -1.4762e-01, -3.0014e-02],
        [ 2.3002e+01,  1.1627e+01, -2.4453e+00,  5.8197e-01,  3.0398e+00,
          9.8343e-01,  3.5733e+00, -5.3185e-03,  4.1998e-03],
        [-2.7175e+01, -3.3500e+01, -2.2698e+00,  5.7747e-01,  3.0104e+00,
          9.5455e-01,  4.1991e+00, -2.2047e-01,  1.0022e-01],
        [ 2.0043e+01, -3.7957e+01, -2.3458e+00,  5.3423e-01,  2.8355e+00,
          9.1554e-01,  1.9321e+00,  6.5049e-03, -6.1924e-03],
        [ 2.2026e+01,  1.3543e+01, -2.4488e+00,  6.2918e-01,  2.7597e+00,
          1.0144e+00,  3.6989e+00, -2.0714e-03,  6.5082e-03],
        [-2.2086e+01,  3.6464e+01, -2.4875e+00,  5.7616e-01,  3.2172e+00,
          8.8926e-01,  1.9890e+00,  3.4211e-02, -1.7238e-02],
        [-2.8063e+01,  3.2998e+01, -2.4040e+00,  5.5324e-01,  3.1785e+00,
          9.2771e-01,  5.2393e+00,  1.5777e-02, -4.2070e-02],
        [-1.9637e+01, -3.7935e+01, -2.2329e+00,  6.0938e-01,  2.8710e+00,
          9.5362e-01,  1.1403e+00, -1.1114e-02,  1.9751e-02],
        [-2.4089e+01,  3.5444e+01, -2.4623e+00,  5.7124e-01,  3.1284e+00,
          9.0833e-01,  1.9569e+00,  4.2764e-02, -1.3171e-02],
        [ 2.0104e+01, -1.6167e+01, -2.4279e+00,  5.8103e-01,  1.8538e+00,
          9.9792e-01,  2.6295e+00,  6.9754e-03, -6.7597e-03],
        [ 1.5569e+01, -1.3036e+00, -2.3180e+00,  5.7348e-01,  1.6674e+00,
          1.1259e+00,  1.4371e+00, -1.5281e-02,  5.1808e-03],
        [ 4.2471e+01,  1.8102e+00, -2.7972e+00,  5.7331e-01,  3.4706e+00,
          9.2078e-01, -3.6133e-02, -9.4383e-02, -1.2933e-01],
        [ 1.4586e+01, -2.1324e-01, -2.3912e+00,  5.3125e-01,  2.5575e+00,
          1.0076e+00,  3.2714e+00, -1.0308e-02, -1.3036e-03],
        [ 2.2508e+01, -3.6480e+01, -2.4238e+00,  5.7857e-01,  2.7187e+00,
          9.4511e-01,  1.9594e+00, -1.1421e-03, -1.2487e-02],
        [ 1.4152e+01, -2.5262e-01, -2.3843e+00,  5.2527e-01,  2.6449e+00,
          1.0262e+00,  3.2640e+00, -3.2033e-03,  9.8183e-03],
        [ 1.1189e+01,  2.3081e+01, -2.4391e+00,  6.2530e-01,  2.7488e+00,
          9.5248e-01,  1.1042e+00,  1.3517e-01, -3.9946e-02],
        [-3.8484e+01, -1.9047e+01, -2.3601e+00,  6.2874e-01,  3.7105e+00,
          9.1644e-01,  4.3859e+00,  2.1103e-02, -1.6693e-03],
        [-2.6121e+01,  3.4021e+01, -2.4488e+00,  5.5526e-01,  3.1969e+00,
          9.3174e-01,  2.0386e+00,  9.5349e-03, -3.3057e-02],
        [-3.5015e+01,  2.4566e+01, -2.5109e+00,  4.8882e-01,  3.0116e+00,
          9.8433e-01,  2.2306e+00,  1.7899e-02,  1.0813e-03],
        [ 2.1520e+01, -1.4121e+01, -2.3904e+00,  5.8005e-01,  2.7689e+00,
          9.2393e-01,  2.6301e+00,  6.3998e-03, -4.4765e-03],
        [-3.7939e+01, -2.0541e+01, -2.2396e+00,  5.8170e-01,  3.7094e+00,
          9.1540e-01,  4.3378e+00,  1.8178e-02,  6.5855e-03],
        [ 4.2469e+01,  4.7610e+00, -2.8298e+00,  6.5316e-01,  3.3970e+00,
          9.7481e-01,  6.3122e-03,  5.2391e-03,  4.4072e-04],
        [-3.7873e+01,  2.0545e+01, -2.3488e+00,  5.3970e-01,  3.5391e+00,
          9.2208e-01,  2.0886e+00,  2.3495e-02, -3.8745e-03],
        [-3.0074e+01, -3.0073e+01, -2.2586e+00,  5.4464e-01,  3.1688e+00,
          9.2537e-01,  1.0418e+00,  1.6906e-02,  5.0770e-03],
        [ 1.4622e+01, -1.3443e+00, -2.3087e+00,  6.3741e-01,  1.5936e+00,
          1.2277e+00,  1.4217e+00, -2.5306e-02,  1.6148e-02],
        [ 2.5065e+01, -3.4936e+01, -2.4312e+00,  6.9418e-01,  1.8843e+00,
          1.0308e+00,  2.0315e+00, -4.1025e-03, -2.2320e-02],
        [-2.0630e+01,  3.7433e+01, -2.4216e+00,  5.9046e-01,  3.1495e+00,
          9.2528e-01,  2.0444e+00,  1.9255e-02, -2.4842e-02],
        [-3.9437e+01, -1.8635e+01, -2.3270e+00,  5.9516e-01,  3.5635e+00,
          9.6302e-01,  4.8818e+00,  4.4709e-03,  2.3243e-03],
        [-3.3513e+01, -2.7504e+01, -2.3177e+00,  6.0947e-01,  3.4579e+00,
          9.8655e-01,  1.1294e+00,  1.2083e-02,  8.9718e-03],
        [-3.5464e+01, -2.6056e+01, -2.2326e+00,  5.5686e-01,  3.2209e+00,
          9.7496e-01,  4.4734e+00,  1.2818e-03,  7.2538e-03],
        [-3.2599e+01, -2.7093e+01, -2.3016e+00,  5.9345e-01,  3.1852e+00,
          9.3854e-01,  1.0317e+00,  2.0024e-02,  3.7644e-03],
        [-2.9995e+01,  3.0495e+01, -2.3979e+00,  5.4502e-01,  3.2967e+00,
          9.4862e-01,  2.0310e+00,  1.3737e-02, -1.1796e-04],
        [ 1.9151e+01, -1.7115e+01, -2.3596e+00,  6.0948e-01,  1.9101e+00,
          9.8393e-01,  2.7945e+00, -2.5399e-03, -6.6870e-03],
        [-2.7018e+01,  3.3915e+01, -2.3954e+00,  5.9589e-01,  3.0610e+00,
          9.6678e-01,  2.0641e+00, -5.4920e-02, -5.7487e-02],
        [ 2.2496e+01, -1.2609e+01, -2.3877e+00,  5.3464e-01,  2.7452e+00,
          9.1884e-01,  2.7467e+00,  3.6692e-03, -2.2370e-03],
        [-3.1477e+01, -2.9991e+01, -2.2319e+00,  5.9286e-01,  2.9733e+00,
          9.7022e-01,  1.1848e+00,  1.5937e-02,  1.0641e-03],
        [-2.8484e+01, -3.2916e+01, -2.2319e+00,  6.8264e-01,  1.8245e+00,
          1.0562e+00,  1.0224e+00, -8.3729e-01,  3.6148e-01],
        [-2.4995e+01, -6.1494e+00, -2.4971e+00,  6.0711e-01,  2.7730e+00,
          8.9183e-01,  3.4085e+00,  2.4583e-01, -1.0499e+00],
        [ 1.1081e+01, -2.7887e+00, -2.1890e+00,  6.6521e-01,  2.9759e+00,
          1.1768e+00,  3.2165e+00, -9.2059e-03, -4.2270e-04],
        [-1.8653e+01,  3.8440e+01, -2.3694e+00,  5.9890e-01,  2.8937e+00,
          9.4022e-01,  2.0365e+00,  1.2065e-02, -1.0770e-02]])), 'scores_3d': tensor([0.8836, 0.4134, 0.2342, 0.0913, 0.0820, 0.0622, 0.0552, 0.0552, 0.0525,
        0.0508, 0.0504, 0.6895, 0.2632, 0.2589, 0.0838, 0.0699, 0.1294, 0.0520,
        0.0644, 0.0509, 0.0773, 0.0605, 0.0584, 0.1063, 0.0531, 0.1318, 0.1069,
        0.1060, 0.1036, 0.0983, 0.0958, 0.0885, 0.0866, 0.0814, 0.0809, 0.0806,
        0.0802, 0.0780, 0.0771, 0.0764, 0.0750, 0.0746, 0.0741, 0.0737, 0.0688,
        0.0679, 0.0674, 0.0672, 0.0670, 0.0657, 0.0653, 0.0646, 0.0645, 0.0624,
        0.0605, 0.0603, 0.0601, 0.0594, 0.0585, 0.0579, 0.0577, 0.0565, 0.0564,
        0.0559, 0.0550, 0.0543, 0.0532, 0.0521, 0.0520, 0.0519, 0.0518, 0.0518,
        0.0517, 0.0516, 0.0508, 0.0501, 0.0501]), 'labels_3d': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 2, 2, 3, 3, 4, 4, 7, 7, 7, 8,
        8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9,
        9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9,
        9, 9, 9, 9, 9])}}]